# Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

- если источник traffic_source равен yandex или google, то в source_type ставится organic
- для источников paid и email из России - ставим ad
- для источников paid и email не из России - ставим other
- все остальные варианты берем из traffic_source без изменений

In [7]:
import pandas as pd

log=pd.read_csv('/Users/socks/netology/PYDA-22/Files/visit_log_pandas.csv', sep=';')
log.head()

def filtered(row):
    if row['traffic_source']=='yandex':
        return 'organic'
    elif row['traffic_source']=='google':
        return 'organic'
    elif row['traffic_source']=='paid':
        if row['region']=='Russia':
            return 'ad'
        else:
            return 'other'
    elif row['traffic_source']=='email':
        if row['region']=='Russia':
            return 'ad'
        else:
            return 'other'
    else:
        return row['traffic_source']
    
log['source_type']=log.apply(filtered, axis=1)
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


# Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

Прочитайте содержимое файла с датафрейм
Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [11]:
import pandas as pd
import re

urls=pd.read_csv('/Users/socks/netology/PYDA-22/Files/URLs.txt')
urls[urls.url.str.contains('/[0-9]{8}-', regex=True)].head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


# Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [27]:
import pandas as pd
ratings=pd.read_csv('/Users/socks/netology/PYDA-22/Files/ml-latest-small/ratings.csv', sep=',')

# находим пользователей выставивших более 100 оценок
ratings_count=ratings.groupby('userId').count().reset_index()
ratings_count=ratings_count[['userId','rating']]

count_more_than_100=ratings_count[ratings_count['rating']>100]
count_more_than_100.head()

,userId,rating
3,4,204
7,8,116
14,15,1700
16,17,363
18,19,423


In [28]:
rating_max=ratings.groupby('userId').max().reset_index()
rating_min=ratings.groupby('userId').min().reset_index()

rating_max_timestamp=rating_max[['userId','timestamp']]
rating_min_timestamp=rating_min[['userId','timestamp']]

rating_max_timestamp.rename(columns={'timestamp': 'timestamp_max'}, inplace=True)
rating_min_timestamp.rename(columns={'timestamp': 'timestamp_min'}, inplace=True)

avg_timestamp=count_more_than_100.merge(rating_max_timestamp, how='inner').merge(rating_min_timestamp, how='inner')

def average(row):
    mean_=(row['timestamp_max']-row['timestamp_min'])
    return mean_

avg_timestamp['avg']=avg_timestamp.apply(average, axis=1)
len(count_more_than_100)==len(avg_timestamp)

avg_timestamp.head()

/Users/socks/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,userId,rating,timestamp_max,timestamp_min,avg
0,4,204,949982274,949778714,203560
1,8,116,1154474527,1154389340,85187
2,15,1700,1469330735,997937239,471393496
3,17,363,1127476640,1127468587,8053
4,19,423,855195373,855190091,5282


# Задание 4

Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе Материалы для лекции «Продвинутый pandas» ---- Ноутбуки к лекции «Продвинутый pandas»).

Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типамвыручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:

- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [36]:
import pandas as pd

rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [37]:
from typing import List
from functools import reduce

def merge_dfs(dfs: List[pd.DataFrame], **kwargs) -> pd.DataFrame:
    return reduce(lambda res, df: res.merge(df, **kwargs), dfs)

merged_data = merge_dfs([client_base, rzd, auto, air], on='client_id', how='outer')

# первая таблица
merged_data.drop(columns='address')

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [43]:
# вторая таблица
merged_data

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0
